In [ ]:
# pip install torchmetrics
# pip install stepmix
# pip install kneed

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
from joblib import Parallel, delayed # for parallelization
from itertools import product

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Clustering
from sklearn.cluster import KMeans, AgglomerativeClustering, HDBSCAN
from stepmix.stepmix import StepMix

# Evaluation
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import torch
from torchmetrics.clustering import DunnIndex
from kneed import KneeLocator

# Data, parameters and validity indexes

In [2]:
data2004_i = pd.read_parquet("data/data2004_i.parquet") # load imputed data

# Dataset with numeric outcomes
data_n = data2004_i[[
    'clseusa_n', 'ambornin_n', 'amcit_n', 'amlived_n', 'amenglsh_n', 
     'amchrstn_n', 'amgovt_n', 'amfeel_n', 'amcitizn_n', 'amshamed_n', 
     'belikeus_n', 'ambetter_n', 'ifwrong_n', 'proudsss_n', 'proudgrp_n', 
     'proudpol_n', 'prouddem_n', 'proudeco_n', 'proudspt_n', 'proudart_n', 
     'proudhis_n', 'proudmil_n', 'proudsci_n']]

# Dataset with categorical outcomes
data_f = data2004_i[[
     'clseusa_f', 'ambornin_f', 'amcit_f', 'amlived_f', 'amenglsh_f', 
     'amchrstn_f', 'amgovt_f', 'amfeel_f', 'amcitizn_f', 'amshamed_f', 
     'belikeus_f', 'ambetter_f', 'ifwrong_f', 'proudsss_f', 'proudgrp_f', 
     'proudpol_f', 'prouddem_f', 'proudeco_f', 'proudspt_f', 'proudart_f', 
     'proudhis_f', 'proudmil_f', 'proudsci_f']]

# Dataset with controls
controls = data2004_i[[
    'sex', 'race_f', 'born_usa', 'party_fs', 'religstr_f', 
    'reltrad_f', 'region_f']]

In [3]:
max_clust = 12

In [4]:
# Custom score functions to avoid throwing errors when undefined
def sil_score(data, pred_clust):
    try:
        sil_score = silhouette_score(data, pred_clust)
    except ValueError:
        sil_score = np.nan
    return sil_score

def ch_score(data, pred_clust):
    try:
        ch_score = calinski_harabasz_score(data, pred_clust)
    except ValueError:
        ch_score = np.nan
    return ch_score

def db_score(data, pred_clust):
    try:
        db_score = davies_bouldin_score(data, pred_clust)
    except ValueError:
        db_score = np.nan
    return db_score

def dunn_score(data, pred_clust):
    torch_data = np.array(data)
    torch_data = torch.tensor(torch_data, dtype=torch.float32)
    torch_pred_clust = torch.tensor(pred_clust, dtype=torch.int64)

    dunn_metric = DunnIndex()
    
    try:
        dunn_score = float(dunn_metric(torch_data, torch_pred_clust))
    except Exception:
        dunn_score = np.nan
 
    return dunn_score

def inertia(data, labels):
    data = np.asarray(data)
    
    inertia = 0
    for cluster in np.unique(labels):
        cluster_points = data[labels == cluster]
        cluster_centroid = np.mean(cluster_points, axis=0)
        inertia += np.sum((cluster_points - cluster_centroid) ** 2)
        
    return inertia

# Latent models
With the StepMix package

Documentation : https://github.com/Labo-Lacourse/stepmix

In [5]:
clust_range = range(1, max_clust)

## Without covariates

In [6]:
def do_StepMix(n, type, data):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)

        model = StepMix(
            n_components = n, 
            measurement = type, 
            n_init = 3)
        
        model.fit(data)
        pred_clust = model.predict(data)

        return {
            'model': 'LCA' if type == 'categorical' else 'LPA',
            'params': 'no covariates',
            'n_clust': n,
            'aic': model.aic(data),
            'bic': model.bic(data),
            'silhouette': sil_score(data, pred_clust),
            'calinski_harabasz': ch_score(data, pred_clust),
            'davies_bouldin': db_score(data, pred_clust),
            'dunn': dunn_score(data, pred_clust),
            'inertia': inertia(data, pred_clust)
        }

data = data_f.apply(lambda col: LabelEncoder().fit_transform(col))
cat_res = Parallel(n_jobs=8)(delayed(do_StepMix)(n, 'categorical', data) for n in clust_range)
LCA_res = pd.DataFrame(cat_res)

num_res = Parallel(n_jobs=8)(delayed(do_StepMix)(n, 'continuous', data_n) for n in clust_range)
LPA_res = pd.DataFrame(num_res)

Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...


Initializations (n_init) : 100%|██████████| 3/3 [00:00<00:00,  9.04it/s, max_LL=-2.82e+4, max_avg_LL=-23.2]


Fitting StepMix...
Fitting StepMix...


Initializations (n_init) : 100%|██████████| 3/3 [00:00<00:00, 28.62it/s, max_LL=-5.65e+3, max_avg_LL=-4.65]


Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...
Fitting StepMix...


## With covariates

In [ ]:
def do_StepMix_covar(n, type, data):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)

        model = StepMix(
            n_components = n, 
            measurement = type, 
            n_init = 3,
            n_steps = 1,
            structural = 'covariate', 
            structural_params = opt_params,
            init_params = 'kmeans',
            random_state = 123)
        
        model.fit(data, controls_dum)
        pred_clust = model.predict(data)

        return {
            'model': 'LCA' if type == 'categorical' else 'LPA',
            'params': 'with covariates',
            'n_clust': n,
            'aic': model.aic(data),
            'bic': model.bic(data),
            'silhouette': sil_score(data, pred_clust),
            'calinski_harabasz': ch_score(data, pred_clust),
            'davies_bouldin': db_score(data, pred_clust),
            'dunn': dunn_score(data, pred_clust),
            'inertia': inertia(data, pred_clust)
        }

opt_params = {
    'method': 'gradient',
    'intercept': True,
    'max_iter': 2500,
}

controls_dum = pd.get_dummies(controls)

data = data_f.apply(lambda col: LabelEncoder().fit_transform(col))
cat_res = Parallel(n_jobs=8)(delayed(do_StepMix_covar)(n, 'categorical', data) for n in clust_range)
LCA_covar_res = pd.DataFrame(cat_res)

num_res = Parallel(n_jobs=8)(delayed(do_StepMix_covar)(n, 'continuous', data_n) for n in clust_range)
LPA_covar_res = pd.DataFrame(num_res)

## Best latent models

In [ ]:
# Find best models according to absolute fit = min aic / bic

In [ ]:
# Find best models according to relative fit = LRT / BLRT / BVR (LCA only)

# K-means

In [7]:
scaler = StandardScaler()
data = scaler.fit_transform(data_n)

results = []

def do_kmeans(n): 
    kmeans = KMeans(
        n_clusters = n, 
        init = 'k-means++', 
        n_init = 25,
        random_state=42)

    pred_clust = kmeans.fit_predict(data)
            
    return{
        'model': 'kmeans',
        'params': 'centroid',
        'n_clust': n,
        'silhouette': sil_score(data, pred_clust),
        'calinski_harabasz': ch_score(data, pred_clust),
        'davies_bouldin': db_score(data, pred_clust),
        'dunn': dunn_score(data, pred_clust),
        'inertia': inertia(data, pred_clust)
    }   

clust_range = range(1, max_clust)

results = Parallel(n_jobs=8)(delayed(do_kmeans)(n) for n in clust_range)

kmeans_res = pd.DataFrame(results)

In [8]:
# Add other models, which are not implemented in sklearn

In [9]:
best_silhouette = kmeans_res.sort_values('silhouette', ascending=False).iloc[0]
best_calinski = kmeans_res.sort_values('calinski_harabasz', ascending=False).iloc[0]
best_davies = kmeans_res.sort_values('davies_bouldin', ascending=True).iloc[0] # Lower is better

# AHC

In [11]:
scaler = StandardScaler()
data = scaler.fit_transform(data_n)

results = []

def do_AHC(n, dist, linkage):
    ahc = AgglomerativeClustering(
        n_clusters = n,
        metric = dist,
        linkage = linkage)
    
    ahc.fit(data)
    
    pred_clust = ahc.labels_

    return {
        'model': 'AHC',
        'params': f"distance = {dist}, linkage = {linkage}",
        'n_clust': n,
        'silhouette': sil_score(data, pred_clust),
        'calinski_harabasz': ch_score(data, pred_clust),
        'davies_bouldin': db_score(data, pred_clust),
        'dunn': dunn_score(data, pred_clust),
        'inertia': inertia(data, pred_clust)
    }

clust_range = range(1, max_clust)
distances = ['manhattan', 'euclidean', 'chebyshev']
linkages = ['single', 'average', 'complete']
params = product(clust_range, distances, linkages)

results = Parallel(n_jobs=8)(delayed(do_AHC)(n, dist, linkage) for n, dist, linkage in params)

results.extend([do_AHC(n, 'euclidean', 'ward') for n in clust_range])

ahc_res = pd.DataFrame(results)

# HDBSCAN

In [14]:
scaler = StandardScaler()
data = scaler.fit_transform(data_n)

results = []

def do_hdbscan(dist, min_c, min_s):
    hdb = HDBSCAN(
        metric = dist,
        min_cluster_size = min_c, 
        min_samples = min_s)
        
    pred_clust = hdb.fit_predict(data)
        
    n_clusters = len(set(pred_clust[pred_clust != -1]))
    noise_freq = 100 * sum(pred_clust == -1) / len(pred_clust)
        
    return {
        'model': 'HDBSCAN',
        'params': f"distance = {dist}, min_cluster_size = {min_c}, min_samples = {min_s}",
        'n_clust': n_clusters,
        'noise': noise_freq,
        'silhouette': sil_score(data, pred_clust),
        'calinski_harabasz': ch_score(data, pred_clust),
        'davies_bouldin': db_score(data, pred_clust),
        'dunn': dunn_score(data, pred_clust),
        'inertia': inertia(data, pred_clust)
    }

distances = ['euclidean', 'chebyshev']
min_cluster_sizes = range(2, 16)
min_samples_range = range(1, 16)
params = product(distances, min_cluster_sizes, min_samples_range)

results = Parallel(n_jobs=8)(delayed(do_hdbscan)(dist, min_c, min_s) for dist, min_c, min_s in params)

hdbscan_res = pd.DataFrame(results)

In [ ]:
best_silhouette = hdbscan_res.sort_values('silhouette', ascending=False).iloc[0]
best_calinski = hdbscan_res.sort_values('calinski_harabasz', ascending=False).iloc[0]
best_davies = hdbscan_res.sort_values('davies_bouldin', ascending=True).iloc[0]

In [ ]:
hdbscan_res['n_clust'].unique()

# Aggregate and display results

In [17]:
res_df = [LCA_res, LPA_res, kmeans_res, ahc_res, hdbscan_res]
# add LCA_covar_res and LPA_covar_res for the final run

combined_res = pd.concat(res_df, ignore_index=True)
combined_res = combined_res.reset_index(drop=True)

In [ ]:
# Display the optimal numbers of clutsters according to validity indexes
def elbow_plot(df, val_index):
    res = df.dropna(subset=[val_index])

    x = res["n_clust"]
    y = res[val_index]

    if metric == 'davies_bouldin':
        knee_locator = KneeLocator(x, y, curve='concave', direction='increasing')
    else:
        knee_locator = KneeLocator(x, y, curve='convex', direction='decreasing')

    plt.figure(figsize=(8, 5))
    plt.plot(x, y, marker="o", linestyle="-", label=val_index)
    plt.axvline(x=knee_locator.knee, color="r", linestyle="--", label=f"Optimal k={knee_locator.knee}")
    plt.xlabel("Number of Clusters")
    plt.ylabel(f"{val_index} index")
    plt.title(f"Elbow Method for {val_index} index")
    plt.legend()
    plt.show()

for val_index in ('silhouette','calinski_harabasz', 'davies_bouldin', 'dunn', 'inertia'):
    elbow_plot(LPA_res, val_index)

In [19]:
# Find best models according to validity indexes
best_models = pd.DataFrame()

def elbow_method(df, val_index):
    res = df.dropna(subset=[val_index])

    x = res["n_clust"]
    y = res[val_index]

    if val_index == 'davies_bouldin':
        knee_locator = KneeLocator(x, y, curve='concave', direction='increasing')
    else:
        knee_locator = KneeLocator(x, y, curve='convex', direction='decreasing')
    
    return res[res["n_clust"] == knee_locator.knee]

models = [LCA_res, LPA_res, kmeans_res, ahc_res, hdbscan_res]
val_indexes = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn', 'inertia']
params = product(models, val_indexes)

for model, val_index in params:
    best_model = elbow_method(model, val_index)
    best_models = pd.concat([best_models, best_model], ignore_index=True)

In [ ]:
# Should work in 2 steps:
## apply knee locator to each unique combination of model and params
## then keep the best model, for each validity indexes, for each class of model

In [20]:
# Keep unique models
best_models.drop_duplicates().reset_index(drop=True)
# In the best_models df, have as many columns has performance criteria, and use dummy for each that is maximized

,model,params,n_clust,aic,bic,silhouette,calinski_harabasz,davies_bouldin,dunn,inertia,noise
0,LCA,no covariates,3,56955.449635,58373.944456,0.050062,126.102058,3.165817,0.195118,24374.255479,NaN
1,LCA,no covariates,6,55352.664858,58194.756999,0.031038,71.040141,3.630874,0.224198,22759.576952,NaN
2,LCA,no covariates,4,56157.435618,58050.462879,0.050236,104.346795,3.006889,0.228913,23397.961936,NaN
3,LPA,no covariates,5,31566.696483,32760.681332,0.007748,64.007200,4.076536,0.152632,24463.602761,NaN
4,LPA,no covariates,3,42378.838560,43093.188470,0.071582,92.974102,3.560600,0.225306,25697.394253,NaN
...,...,...,...,...,...,...,...,...,...,...,...
238,HDBSCAN,"distance = chebyshev, min_cluster_size = 15, m...",2,NaN,NaN,-0.052586,57.014071,3.486657,0.178472,25541.946622,60.823045
239,HDBSCAN,"distance = chebyshev, min_cluster_size = 15, m...",2,NaN,NaN,-0.055658,56.480595,3.389945,0.185127,25562.514767,61.810700
240,HDBSCAN,"distance = chebyshev, min_cluster_size = 15, m...",2,NaN,NaN,-0.055591,56.415313,3.375179,0.177203,25565.034015,62.469136
241,HDBSCAN,"distance = chebyshev, min_cluster_size = 15, m...",2,NaN,NaN,-0.062728,54.729870,3.421150,0.166293,25630.247340,63.292181


# Clusters visualization 

In [ ]:
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull

In [ ]:
# PCA to represent the clusters in 2D
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(data)

## For kmeans

In [ ]:
# Fit an arbitrary model
scaler = StandardScaler()
data = scaler.fit_transform(data_n)

kmeans = KMeans(n_clusters=7, random_state=42)
pred_clust = kmeans.fit_predict(data)

### Datapoints alone

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=pred_clust, cmap='tab10', s=20, edgecolors='k')
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.axhline(y=0, color='#333333', linestyle='--', linewidth=1)
plt.axvline(x=0, color='#333333', linestyle='--', linewidth=1)
plt.title("Clusters")
plt.show()

### With decision boundaries

In [ ]:
# Create a grid for boundary visualization in 2D space
x_min, x_max = X_reduced[:, 0].min() - 0.5, X_reduced[:, 0].max() + 0.5
y_min, y_max = X_reduced[:, 1].min() - 0.5, X_reduced[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))

# Project grid points back to original space
grid_points_2D = np.c_[xx.ravel(), yy.ravel()]
grid_points_original = pca.inverse_transform(grid_points_2D)

# Predict clusters in the original space
grid_clusters = kmeans.predict(grid_points_original).reshape(xx.shape)

plt.figure(figsize=(8, 6))

# Create scatter plot first to get the color mapping
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                     c=pred_clust, cmap='tab10', 
                     s=15, edgecolors='k')

# Plot boundaries using the same colormap and normalization
plt.contourf(xx, yy, grid_clusters, 
             alpha=0.3, 
             cmap=scatter.cmap,
             norm=scatter.norm)

# Plot centroids with labels
centroids_pca = pca.transform(kmeans.cluster_centers_)
for i, (x, y) in enumerate(centroids_pca):
    plt.text(x, y, str(i), color='white', fontsize=12, 
             ha='center', va='center', fontweight='bold',
             bbox=dict(facecolor='black', edgecolor='none', boxstyle='round,pad=0.2'))

plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.title("Clusters with Decision Boundaries")
plt.show()

### With convex hulls

In [ ]:
plt.figure(figsize=(8, 6))

# Collect all hull vertices
hull_vertices = []
hull_colors = []
for i in range(kmeans.n_clusters):
    cluster_points = X_reduced[pred_clust == i]
    if len(cluster_points) > 2:
        hull = ConvexHull(cluster_points)
        hull_vertices.append((
            cluster_points[hull.vertices, 0],
            cluster_points[hull.vertices, 1]
        ))
        hull_colors.append(i)

# Plot datapoints
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                     c=pred_clust, cmap='tab10', 
                     s=15, edgecolors='k')

# Plot all hulls using the same colormap
for vertices, i in zip(hull_vertices, hull_colors):
    plt.fill(vertices[0], vertices[1], 
             alpha=0.3,
             color=scatter.cmap(scatter.norm(i)))

legend = plt.legend(*scatter.legend_elements())
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.title("Clusters with Convex Hulls")
plt.show()

## For HDBSCAN
Example of non-convex clusters in the PCA space

In [ ]:
# Fit an arbitrary model
scaler = StandardScaler()
data = scaler.fit_transform(data_n)

hdb = HDBSCAN(min_cluster_size = 5, min_samples = 1)  
pred_clust = hdb.fit_predict(data)
n_clusters = len(set(pred_clust[pred_clust != -1]))

In [ ]:
plt.figure(figsize=(8, 6))

# Collect all hull vertices
hull_vertices = []
hull_colors = []
for i in range(n_clusters):
    cluster_points = X_reduced[pred_clust == i]
    if len(cluster_points) > 2:
        hull = ConvexHull(cluster_points)
        hull_vertices.append((
            cluster_points[hull.vertices, 0],
            cluster_points[hull.vertices, 1]
        ))
        hull_colors.append(i)

# Plot datapoints
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                     c=pred_clust, cmap='tab10', 
                     s=15, edgecolors='k')

# Plot all hulls using the same colormap
for vertices, i in zip(hull_vertices, hull_colors):
    plt.fill(vertices[0], vertices[1], 
             alpha=0.7,
             color=scatter.cmap(scatter.norm(i)))

legend = plt.legend(*scatter.legend_elements())
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.title("Clusters with Convex Hulls")
plt.show()